# Tasca M10 T01

## Exercici 1
Realitza web scraping de dues de les tres pàgines web proposades utilitzant BeautifulSoup primer i Selenium després. 

- http://quotes.toscrape.com

- https://www.bolsamadrid.es

- www.wikipedia.es (fes alguna cerca primer i escrapeja algun contingut)



In [1]:
#!pip install beautifulsoup4

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import requests
#import jovian
from bs4 import BeautifulSoup
import csv
import string 
from datetime import datetime
import warnings
warnings.filterwarnings("ignore")

### Se utilizará esta página http://quotes.toscrape.com/ para recuperar la información mediante web scraping con BeautifulSoup.


**Quotes to Scrape** es una página web de citas populares de diferentes autores relacionados con diferentes temas. 


- Se realizará web scraping a citas con sus autores, etiquetas (temas) y enlaces a la biografía del autor.

- ¿Qué es el raspado web (Web Scraping)?
Web Scraping es el proceso de extracción y análisis de datos de una página web de forma automatizada mediante un programa informático. Hay muchos ejemplos en los que el web scraping puede ser útil, por ejemplo, para crear un raspador de búsqueda de empleo, un raspador de películas principales, un raspador de precios de teléfonos móviles, etc. 
El raspado web implica extraer información a menudo de documentos HTML y luego convertirla en datos estructurados como archivos CSV .

- Se descargará la página web Quotes To Scrape usando la biblioteca **Requests**
***Requests** es una biblioteca HTTP (protocolo de transferencia de hipertexto) elegante y simple para Python que le permite enviar solicitudes HTTP fácilmente. **Requests.get** descargará la página web y devolverá un objeto de respuesta con el contenido de la página y alguna información que indique si la solicitud fue exitosa, usando un código de estado.

In [3]:
url='http://quotes.toscrape.com/'
response=requests.get(url)

- **response.status_code** le proporcionará el código si la solicitud fue exitosa o no. Si **status.code** se encuentra entre 200 y 209, la solicitud fue exitosa; de lo contrario, no fue exitosa.

In [4]:
response.status_code

200

- La solicitud fue exitosa


### Análisis de partes del sitio web usando BeautifulSoup

- Para extraer información del código fuente HTML mediante programación, se utilizará la biblioteca **Beautiful Soup**.

- Beautiful Soup devolverá un objeto que contiene varias propiedades y métodos para extraer la información de los documentos HTML.

In [5]:
doc=BeautifulSoup(response.text,'html.parser')
type(doc)

bs4.BeautifulSoup

- Al inspeccionar el cuadro que contiene la información de las citas (quote), encontraremos una divetiqueta para cada cita (quote), con la clase establecida en quote.

- Se buscarán todas las **div** etiquetas que tengan **class** quote.

In [6]:
div_tags=doc.find_all('div',class_='quote')
len(div_tags)

10

- Cada página tiene 10 citas, por lo tanto, el **length** de **div_tags** es 10. 
- **div_tags** Contiene información como cita, nombre del autor, enlaces a la biografía del autor, etiquetas.
- Se extraerán las comillas usando la función **get_quotes**.

In [7]:
def get_quotes(div_tags):
    """Get the list of quotes for one page"""
    quotes=[]
    for tag in div_tags:
        quote=tag.find('span',class_='text').text
        quotes.append(quote)
    return quotes    

In [8]:
get_quotes(div_tags)

['“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”',
 '“It is our choices, Harry, that show what we truly are, far more than our abilities.”',
 '“There are only two ways to live your life. One is as though nothing is a miracle. The other is as though everything is a miracle.”',
 '“The person, be it gentleman or lady, who has not pleasure in a good novel, must be intolerably stupid.”',
 "“Imperfection is beauty, madness is genius and it's better to be absolutely ridiculous than absolutely boring.”",
 '“Try not to become a man of success. Rather become a man of value.”',
 '“It is better to be hated for what you are than to be loved for what you are not.”',
 "“I have not failed. I've just found 10,000 ways that won't work.”",
 "“A woman is like a tea bag; you never know how strong it is until it's in hot water.”",
 '“A day without sunshine is like, you know, night.”']

- Se extraerán los nombres de los autores, que están dentro **small tag**, con la clase establecida en **author** que está dentro **span tag** con la clase establecida en **None**, y se usará la función de ayuda **get_author_name**.

In [9]:
def get_author_name(div_tags):
    """Get the author names for the quotes for one page"""
    authors=[]
    for tag in div_tags:
        span_tag=tag.find('span',class_=None)
        author=span_tag.find('small',class_='author').text
        authors.append(author)
    return authors  

In [10]:
get_author_name(div_tags)

['Albert Einstein',
 'J.K. Rowling',
 'Albert Einstein',
 'Jane Austen',
 'Marilyn Monroe',
 'Albert Einstein',
 'André Gide',
 'Thomas A. Edison',
 'Eleanor Roosevelt',
 'Steve Martin']

In [11]:
df = pd.DataFrame()
for quote_element in div_tags:
    quote = quote_element.find("span", class_="text")
    author = quote_element.find("small", class_="author")
    df = df.append({'Quote' : quote.text.strip(),
                        'Author' : author.text.strip()},
                    ignore_index = True)
df.head()

,Quote,Author
0,“The world as we have created it is a process ...,Albert Einstein
1,"“It is our choices, Harry, that show what we t...",J.K. Rowling
2,“There are only two ways to live your life. On...,Albert Einstein
3,"“The person, be it gentleman or lady, who has ...",Jane Austen
4,"“Imperfection is beauty, madness is genius and...",Marilyn Monroe


- Se extraerán las etiquetas de las comillas, que están dentro **meta tag** con el atributo **content** que está dentro **div tag** con la clase establecida en **tags**, y se usará la función **get_quote_tag**.

In [12]:
def get_quote_tag(div_tags):
    """Get the quote tags for different quotes for one page"""
    name_tags=[]
    for tag in div_tags:
        name_tag=tag.find('div',class_='tags').meta['content']
        name_tags.append(name_tag)
    return name_tags

In [13]:
get_quote_tag(div_tags)

['change,deep-thoughts,thinking,world',
 'abilities,choices',
 'inspirational,life,live,miracle,miracles',
 'aliteracy,books,classic,humor',
 'be-yourself,inspirational',
 'adulthood,success,value',
 'life,love',
 'edison,failure,inspirational,paraphrased',
 'misattributed-eleanor-roosevelt',
 'humor,obvious,simile']

- Se extraerán los enlaces de la biografía del autor, que está dentro **a tag** con el atributo **href** que está dentro **span tag** con la clase establecida en **None**, usando la función **get_author_links**.



In [14]:
def get_author_urls(div_tags):
    """ Get the list of urls for one page"""
    author_links=[]
    for tag in div_tags:
        span_tag=tag.find('span',class_=None)
        author_link='http://quotes.toscrape.com'+span_tag.find('a')['href']
        author_links.append(author_link)
    return author_links

In [15]:
get_author_urls(div_tags)

['http://quotes.toscrape.com/author/Albert-Einstein',
 'http://quotes.toscrape.com/author/J-K-Rowling',
 'http://quotes.toscrape.com/author/Albert-Einstein',
 'http://quotes.toscrape.com/author/Jane-Austen',
 'http://quotes.toscrape.com/author/Marilyn-Monroe',
 'http://quotes.toscrape.com/author/Albert-Einstein',
 'http://quotes.toscrape.com/author/Andre-Gide',
 'http://quotes.toscrape.com/author/Thomas-A-Edison',
 'http://quotes.toscrape.com/author/Eleanor-Roosevelt',
 'http://quotes.toscrape.com/author/Steve-Martin']

### Ahora se utilizará esta página http://quotes.toscrape.com/ para recuperar la información mediante web scraping con Selenium.

In [16]:
#!pip install -U selenium

In [17]:
#!pip install webdriver-manager

In [18]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time

In [19]:
PATH = "C:\Program Files (x86)\chromedriver.exe"

driver = webdriver.Chrome(PATH)

driver.get("http://quotes.toscrape.com/")

time.sleep(5)

#Find Elements
quotes = driver.find_elements(By.CLASS_NAME, "text")
authors = driver.find_elements(By.CLASS_NAME, "author")

In [20]:
for quote in quotes:
    quote= quote.text.strip()
    print(quote)

“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”
“It is our choices, Harry, that show what we truly are, far more than our abilities.”
“There are only two ways to live your life. One is as though nothing is a miracle. The other is as though everything is a miracle.”
“The person, be it gentleman or lady, who has not pleasure in a good novel, must be intolerably stupid.”
“Imperfection is beauty, madness is genius and it's better to be absolutely ridiculous than absolutely boring.”
“Try not to become a man of success. Rather become a man of value.”
“It is better to be hated for what you are than to be loved for what you are not.”
“I have not failed. I've just found 10,000 ways that won't work.”
“A woman is like a tea bag; you never know how strong it is until it's in hot water.”
“A day without sunshine is like, you know, night.”


In [21]:
for author in authors:
    author= author.text.strip()
    print(author)

Albert Einstein
J.K. Rowling
Albert Einstein
Jane Austen
Marilyn Monroe
Albert Einstein
André Gide
Thomas A. Edison
Eleanor Roosevelt
Steve Martin


- Al comparar los métodos de BeautifulSoup con Selenium, podemos concluir que son muy similares pero con Selenium es más fácil y los datos se obtienen de manera más estructurada (separados con espacios).



### Se utilizará Wikipedia (https://es.wikipedia.org/wiki/Inteligencia_artificial) para recuperar la información mediante web scraping con BeautifulSoup.

- Se realizará web scraping a una búsqueda en la página de wikipedia y luego se restreará algún contenido de interés.

In [22]:
url='https://es.wikipedia.org/wiki/Inteligencia_artificial'
response=requests.get(url)

In [23]:
response.status_code

200

In [24]:
doc=BeautifulSoup(response.text,'html.parser')
type(doc)

bs4.BeautifulSoup

In [25]:
soup = BeautifulSoup(response.text, 'lxml')

In [26]:
soup.find('title')

<title>Inteligencia artificial - Wikipedia, la enciclopedia libre</title>

In [27]:
int_art= soup.find(title="AlphaGo")

print('Búsqueda Wikipedia: '+ int_art.text)


Búsqueda Wikipedia: AlphaGo


### Se utilizará Wikipedia  (https://es.wikipedia.org/wiki/Inteligencia_artificial) para recuperar la información mediante web scraping con Selenium.

- Se realizará web scraping a una búsqueda en la página de wikipedia y luego se restreará algún contenido de interés. 

In [28]:
PATH = "C:\Program Files (x86)\chromedriver.exe"

driver = webdriver.Chrome(PATH)

driver.get("https://es.wikipedia.org/wiki/Inteligencia_artificial")

time.sleep(5)

intel_art = driver.find_element("xpath", '//*[@id="mw-content-text"]/div[1]/ul[4]/li[34]/a[1]')

print('Búsqueda Wikipedia: '+ intel_art.text)

driver.quit()

Búsqueda Wikipedia: AlphaGo


- Se logró rastrear dentro de la historia de la inteligencia artificial, un acontecimiento muy importante.  

## Exercici 2
Documenta en un Word el teu conjunt de dades generat amb la informació que tenen els diferents arxius de Kaggle.

 Per saber més

A manera d'exemple del que es demana pots consultar aquest enllaç:

->https://www.kaggle.com/datasets/vivovinco/20212022-football-team-stats.




**URL QUOTES** 

Acerca del conjunto de datos

**Contexto:**
Este conjunto de datos contiene las 10 citas principales de los últimos tiempos.
Información obtenida de la siguiente url:
-  http://quotes.toscrape.com

**Contenido:** 

- 10 filas y 2 columnas.

La descripción de las columnas se muestra a continuación.
-  Citas: Citas (quotes)
-  Autor: Nombre del autor
- Tag: Etiqueta principal donde se ubica la cita 

Fecha de extracción de los datos 12/01/23


**URL WIKIPEDIA INTELIGENCIA ARTIFICIAL**

Acerca del conjunto de datos

**Contexto:**

Este conjunto de datos contiene información acerca de inteligencia artificial.
Información obtenida de la siguiente url: 
-  https://es.wikipedia.org/wiki/Inteligencia_artificial

**Contenido:**
- Concepto
- Sobre la definición del término
- Tipos
- Escuelas de pensamiento
- Historia
- Implicaciones sociales, éticas y filosóficas
- Regulación
- Objetivos
- Críticas
- Aplicaciones de la inteligencia artificial
- Propiedad intelectual de la inteligencia artificial
- En la cultura popular
- Véase también
- Referencias
- Bibliografía
- Enlaces externos

Se rastreó el nombre de un evento de gran importancia en la historia de la inteligencia artificial, **AlphaGo**.

Fecha de extracción de los datos 12/01/23

## Exercici 3

Tria una pàgina web que tu vulguis i realitza web scraping mitjançant la llibreria Selenium primer i Scrapy després. 

In [29]:
#!pip install Scrapy

In [30]:
import scrapy
from scrapy.crawler import CrawlerProcess

Se realizará **Web Scraping** con **Scrapy** de la siguiente página web: http://books.toscrape.com
- En el repositorio de GitHub se podrá encontrar una carpeta llamada "web_scraping" con los archivos correspondientes al rastreo de datos usando el entorno **Scrapy**. 
- También se realizará web scraping de dicha página con la libreria **lxml**.

In [31]:
df = pd.read_csv('output.csv')
df

,title,price,stock
0,A Light in the ...,£51.77,In stock
1,Tipping the Velvet,£53.74,In stock
2,Soumission,£50.10,In stock
3,Sharp Objects,£47.82,In stock
4,Sapiens: A Brief History ...,£54.23,In stock
5,The Requiem Red,£22.65,In stock
6,The Dirty Little Secrets ...,£33.34,In stock
7,The Coming Woman: A ...,£17.93,In stock
8,The Boys in the ...,£22.60,In stock
9,The Black Maria,£52.15,In stock


- Se puede observar una tabla con el nombre, precio y stock de los 20 primeros libros. 

In [32]:
import requests
from lxml import html

In [33]:
resp = requests.get('http://books.toscrape.com/')

In [34]:
resp.status_code

200

In [35]:
res = requests.delete

In [36]:
soup = html.fromstring(resp.content)

Se obtendrán los titulos de los libros que se venden en de la pagina 

In [37]:
for element in soup.xpath('//article[@class="product_pod"]'):
    print(element.xpath('.//h3/a/text()'))

['A Light in the ...']
['Tipping the Velvet']
['Soumission']
['Sharp Objects']
['Sapiens: A Brief History ...']
['The Requiem Red']
['The Dirty Little Secrets ...']
['The Coming Woman: A ...']
['The Boys in the ...']
['The Black Maria']
['Starving Hearts (Triangular Trade ...']
["Shakespeare's Sonnets"]
['Set Me Free']
["Scott Pilgrim's Precious Little ..."]
['Rip it Up and ...']
['Our Band Could Be ...']
['Olio']
['Mesaerion: The Best Science ...']
['Libertarianism for Beginners']
["It's Only the Himalayas"]


- Se realizará web scraping de la página nombrada anteriormente con la librería **Selenium**

In [38]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from time import sleep

In [39]:
driver = webdriver.Chrome() 
#'C:\Program Files (x86)\chromedriver.exe'

In [40]:
driver.get('http://books.toscrape.com/')

In [41]:
def get_books():
    for element in driver.find_elements('xpath', '//article[@class="product_pod"]'):
        title = element.find_element('xpath', './/h3/a').text
        price = element.find_element('xpath', './/div[@class="product_price"]/p[1]').text
        stock = element.find_element('xpath', './/p[@class="instock availability"]').text
        print(title, price, stock)
   
    #next_page = driver.find_element('xpath', '//*(text()="next")')
    #if next_page.get_attribute('href'):
        #next_page.click()
        #sleep(6)
        #get_books()

In [42]:
get_books()

A Light in the ... £51.77 In stock
Tipping the Velvet £53.74 In stock
Soumission £50.10 In stock
Sharp Objects £47.82 In stock
Sapiens: A Brief History ... £54.23 In stock
The Requiem Red £22.65 In stock
The Dirty Little Secrets ... £33.34 In stock
The Coming Woman: A ... £17.93 In stock
The Boys in the ... £22.60 In stock
The Black Maria £52.15 In stock
Starving Hearts (Triangular Trade ... £13.99 In stock
Shakespeare's Sonnets £20.66 In stock
Set Me Free £17.46 In stock
Scott Pilgrim's Precious Little ... £52.29 In stock
Rip it Up and ... £35.02 In stock
Our Band Could Be ... £57.25 In stock
Olio £23.88 In stock
Mesaerion: The Best Science ... £37.59 In stock
Libertarianism for Beginners £51.33 In stock
It's Only the Himalayas £45.17 In stock


- Se puede observar que se realizó un rastreo de los primeros 20 nombres de los libros, su precio y si están en stock.
- La web contiene 50 páginas con 1000 libros en total.